# Import libraries

In [1]:
import numpy
import numpy as np
#import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from PIL import Image
from torch.utils.data.dataset import Dataset
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms
from torchvision import datasets
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

SEED = 1

# CUDA?
cuda = torch.cuda.is_available()

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

## Dataset

In [2]:
# Image transforms
imageTransformation = transforms.Compose([transforms.Grayscale(),transforms.ToTensor()])

# load data set
imagesData = datasets.ImageFolder('PPC/.',imageTransformation)

# Split 
nbTrain = int(0.8*len(imagesData))
nbTest = len(imagesData) - nbTrain
train_data, test_data = torch.utils.data.random_split(imagesData, [nbTrain, nbTest])

In [3]:
batch = int(nbTrain/6)
dataloader_args_train = dict(shuffle=True, batch_size=batch,num_workers=4, pin_memory=True)
dataloader_args_test = dict(shuffle=True, batch_size=nbTest,num_workers=4, pin_memory=True)
train_loader = dataloader.DataLoader(train_data, **dataloader_args_train)
test_loader = dataloader.DataLoader(test_data, **dataloader_args_test)
iterator = iter(test_loader)
img, lb = iterator.next()

# Model 

In [4]:
# One hidden Layer NN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Linear(1024, 20)
        self.fc2 = nn.Linear(20, 3)

    def forward(self, x):
        x = x.view((-1, 1024))
        h = F.relu(self.fc(x))
        h = self.fc2(h)
        return F.log_softmax(h, dim=0) 
    
    
model = Model()
if cuda:
    model.cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-2)

# Learning

In [5]:
EPOCHS = 100
losses = []
model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        data, target = Variable(data), Variable(target)
        
        if cuda:
            data, target = data.cuda(), target.cuda()

        # Init
        optimizer.zero_grad()
        # Predict
        y_pred = model(data) 

         
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().item())
        
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if batch_idx % batch == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                nbTrain,
                100. * batch_idx / batch, 
                loss.cpu().item()), 
                end='')
            
    # display final evaluation for this epoch
    model.eval()
    output = model(img)
    pred = output.data.max(1)[1]
    d = pred.eq(lb.data).cpu()
    accuracy = d.sum().item()/d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
        epoch+1,
        EPOCHS,
        nbTrain, 
        nbTrain,
        100. * batch_idx / batch, 
        loss.cpu().item(),
        accuracy*100,
        end=''))

 Train Epoch: 1/100 [161/161 (23%)]	Loss: 1.311566	 Test Accuracy: 29.2683%
 Train Epoch: 2/100 [161/161 (23%)]	Loss: 1.008050	 Test Accuracy: 36.5854%
 Train Epoch: 3/100 [161/161 (23%)]	Loss: 1.015332	 Test Accuracy: 36.5854%
 Train Epoch: 4/100 [161/161 (23%)]	Loss: 0.746535	 Test Accuracy: 39.0244%
 Train Epoch: 5/100 [161/161 (23%)]	Loss: 0.853198	 Test Accuracy: 39.0244%
 Train Epoch: 6/100 [161/161 (23%)]	Loss: 0.542448	 Test Accuracy: 41.4634%
 Train Epoch: 7/100 [161/161 (23%)]	Loss: 0.942689	 Test Accuracy: 43.9024%
 Train Epoch: 8/100 [161/161 (23%)]	Loss: 0.671042	 Test Accuracy: 36.5854%
 Train Epoch: 9/100 [161/161 (23%)]	Loss: 0.671707	 Test Accuracy: 46.3415%
 Train Epoch: 10/100 [161/161 (23%)]	Loss: 0.593035	 Test Accuracy: 43.9024%
 Train Epoch: 11/100 [161/161 (23%)]	Loss: 0.656171	 Test Accuracy: 39.0244%
 Train Epoch: 12/100 [161/161 (23%)]	Loss: 1.103304	 Test Accuracy: 43.9024%
 Train Epoch: 13/100 [161/161 (23%)]	Loss: 0.794517	 Test Accuracy: 51.2195%
 Train E